**Primeiramente, utilizei seis algoritmos para o treinamento:**

- LogisticRegression(), 
- RandomForestClassifier(), 
- GradientBoostingClassifier(), 
- SVC(), 
- CatBoostClassifier(),
- keras.Sequential()

**Após avaliar o desempenho de cada modelo com hiperparâmetros padrão, escolhi:**
- RandomForestClassifier() 
- CatBoostClassifier() 

**devido ao seu melhor desempenho. Como ambos são baseados em árvores de decisão, também selecionei**
- keras.Sequential()(Redes Neurais Sequenciais) 

**para incluir um modelo com uma abordagem diferente. Esses três modelos serão passados para o algoritmo de otimização de hiperparâmetros.**


- Para otimizar os hiperparâmetros do modelo Random Forest, utilizei o algoritmo RandomSearch. 
- Para a Redes Neurais Sequenciais, utilizei o algoritmo keras_tuner.RandomSearch,ele vem com algoritmos de Otimização Bayesiana, Hiperbanda e Pesquisa Aleatória integrados e também foi projetado para ser fácil para pesquisadores estenderem a fim de experimentar novos algoritmos de pesquisa.
- Para o otimizar o modelo CatBoost, ele tem um proprio otimizador imbutido na classe chamado `CatBoostClassifier.grid_search`.

# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
import keras
import keras_tuner
from keras import layers
from skopt import BayesSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report,recall_score
import warnings
import pickle
import warnings
import joblib
warnings.filterwarnings("ignore")

2024-07-24 13:14:28.841858: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 13:14:29.032683: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 13:14:33.604035: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 13:14:53.529594: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 13:15:31.973591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

# Importacao dos dados

In [2]:
dados_tratados = pickle.load(open('dados/dados_tratados.pickle', 'rb'))

In [3]:
x_treino = dados_tratados['x_treino']
y_treino = dados_tratados['y_treino']
x_teste = dados_tratados['x_teste']
y_teste = dados_tratados['y_teste']

print(x_treino.shape, y_treino.shape, x_teste.shape, y_teste.shape) # x_treino.shape

(59606, 20) (59606,) (15891, 20) (15891,)


# Treinamento do modelo

## Selecionando os modelos 

In [5]:
## criar a rede seqeuncial

model_rede = keras.Sequential()
model_rede.add(layers.Dense(units=40,activation='relu',input_shape=(20,)))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=60,activation='relu'))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=40,activation='relu'))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=1,activation='sigmoid'))
model_rede.summary()
opt = keras.optimizers.SGD(learning_rate=0.01)
model_rede.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy', 'Recall'])
#epochs_hist = model.fit(x_treino, y_treino.values, epochs=100,batch_size=500)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 40)             │           840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         2,460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         2,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,781 (22.58 KB)

 Trainable params: 5,781 (22.58 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
##
modelos = {
    'logistic_regression': LogisticRegression(),
    'random_forest': RandomForestClassifier(),
    'gradient_boost': GradientBoostingClassifier(),
    'svc': SVC(),
    'cat_boost': CatBoostClassifier(),
    'redes_sequencial': model_rede
}

lista_recall = {}
for key,model in modelos.items():
    if key == 'redes_sequencial': 
        epochs_hist = model.fit(x_treino, y_treino.values, epochs=100,batch_size=500,verbose=0)  
        print('O modelo treinado é o:', key)
        y_pred = model.predict(x_teste).squeeze()
        y_pred_redes = (y_pred > 0.5).astype(int)
        lista_recall[key] = recall_score(y_teste,y_pred_redes)
        print(classification_report(y_teste.values, y_pred_redes))  
        print('----------------------------------------------------')
        
    else:
        model = model
        if key == 'cat_boost':
            model_treinado = model.fit(x_treino, y_treino,verbose=0)
        else:
            model_treinado = model.fit(x_treino, y_treino)
        print('O modelo treinado e de:', key)
        print(key, model_treinado.score(x_teste, y_teste))
        print(classification_report(y_teste, model.predict(x_teste)))
        lista_recall[key] = recall_score(y_teste,model.predict(x_teste))
        print('----------------------------------------------------')


O modelo treinado e de: logistic_regression
logistic_regression 0.9821282486942294
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     15518
           1       0.72      0.39      0.51       373

    accuracy                           0.98     15891
   macro avg       0.85      0.70      0.75     15891
weighted avg       0.98      0.98      0.98     15891

----------------------------------------------------
O modelo treinado e de: random_forest
random_forest 0.9889874771883456
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     15518
           1       0.90      0.60      0.72       373

    accuracy                           0.99     15891
   macro avg       0.95      0.80      0.86     15891
weighted avg       0.99      0.99      0.99     15891

----------------------------------------------------
O modelo treinado e de: gradient_boost
gradient_boost 0.9879176892580706
             

**Análise dos Resultados**

Os modelos de Boosting parecem ter um desempenho melhor, sendo os três melhores:

Selecionei os seguintes modelos para fazer a tunagem:
- Cat Boost
- Random Forest

Para ter um modelo diferente, o terceiro modelo será a rede sequencial.

In [7]:
pd.DataFrame(list(lista_recall.items()), columns=['Model', 'Recall']).sort_values(by='Recall', ascending=False)

,Model,Recall
4,cat_boost,0.705094
1,random_forest,0.595174
2,gradient_boost,0.579088
3,svc,0.439678
5,redes_sequencial,0.396783
0,logistic_regression,0.394102


# Tunagem dos modelos

## Cat Boost

In [12]:
## Parametros Cast Boost
parametros_cat = {"iterations": [2, 10, 20, 30, 50, 100, 200, 300, 400, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1],
                    "depth": [1, 3, 5, 7, 9,20,40,50,60,70,80,90,100],
                    'l2_leaf_reg': [1, 3, 5, 7, 9]
                }

grid_cat = RandomizedSearchCV(
            CatBoostClassifier(verbose=0), parametros_cat, verbose=0, scoring="recall"
        )
grid_cat.fit(x_treino, y_treino)

RandomizedSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7f6730227650>,
                   param_distributions={'depth': [1, 3, 5, 7, 9, 20, 40, 50, 60,
                                                  70, 80, 90, 100],
                                        'iterations': [2, 10, 20, 30, 50, 100,
                                                       200, 300, 400, 500],
                                        'l2_leaf_reg': [1, 3, 5, 7, 9],
                                        'learning_rate': [0.1, 0.15, 0.2, 0.4,
                                                          0.6, 0.8, 1]},
                   scoring='recall')

In [ ]:
cat_model = CatBoostClassifier(verbose=0)

parametros_cat = {"iterations": [2, 10, 20, 30, 50, 100, 200, 300, 400, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1],
                    "depth": [1, 3, 5, 7, 9,20,40,50,60,70,80,90,100],
                    'l2_leaf_reg': [1, 3, 5, 7, 9]
                }

grid_cat = cat_model.grid_search(param_grid=parametros_cat,
                                       X=x_treino,
                                       y=y_treino)

In [16]:
grid_cat.keys()

dict_keys(['params', 'cv_results'])

In [22]:
grid_cat['cv_results']['test-Logloss-mean']

[0.4367745714096049,
 0.2838888584669405,
 0.18486685683608908,
 0.12770619012021825,
 0.09333006092375805,
 0.07296905463961208,
 0.06025660375932421,
 0.0516975671313766,
 0.04622805163346866,
 0.04239183127764645,
 0.039952741417662405,
 0.03766901512137954,
 0.0363182947924083,
 0.03507474973679025,
 0.03387306312326049,
 0.03309413280038704,
 0.03243152553518989,
 0.03181707168920283,
 0.03127972911816037,
 0.030861135163456902,
 0.03041188021039361,
 0.030053117482783193,
 0.029819964092775087,
 0.029522255074962298,
 0.029254102299981762,
 0.02898938553223565,
 0.02880254486358044,
 0.0286331381037737,
 0.028473693323916333,
 0.028385548920269135,
 0.028252516367249857,
 0.02810577982240837,
 0.027981732654733776,
 0.02787317901869658,
 0.02777621156037191,
 0.027708830694387926,
 0.027617678670430783,
 0.027520921670204382,
 0.027447289344104887,
 0.027385679054019518,
 0.02726448679444725,
 0.02717486430246903,
 0.027164412423846544,
 0.027094987326358667,
 0.02700540069712964

In [23]:
print('Melhores parametros:',grid_cat['params'],'Score:',grid_cat['cv_results']['test-Logloss-mean']) #bayes_cat.best_params_,bayes_cat.best_score_

Melhores parametros: {'depth': 9, 'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 300} Score: [0.4367745714096049, 0.2838888584669405, 0.18486685683608908, 0.12770619012021825, 0.09333006092375805, 0.07296905463961208, 0.06025660375932421, 0.0516975671313766, 0.04622805163346866, 0.04239183127764645, 0.039952741417662405, 0.03766901512137954, 0.0363182947924083, 0.03507474973679025, 0.03387306312326049, 0.03309413280038704, 0.03243152553518989, 0.03181707168920283, 0.03127972911816037, 0.030861135163456902, 0.03041188021039361, 0.030053117482783193, 0.029819964092775087, 0.029522255074962298, 0.029254102299981762, 0.02898938553223565, 0.02880254486358044, 0.0286331381037737, 0.028473693323916333, 0.028385548920269135, 0.028252516367249857, 0.02810577982240837, 0.027981732654733776, 0.02787317901869658, 0.02777621156037191, 0.027708830694387926, 0.027617678670430783, 0.027520921670204382, 0.027447289344104887, 0.027385679054019518, 0.02726448679444725, 0.02717486430246903, 0.02716

In [ ]:
cat_model = CatBoostClassifier(**grid_cat['params'])
cat_model.fit(x_treino, y_treino)

In [31]:
cat_model.save_model("modelos_salvos/cat_boost_model.cbm")

## Random Forest

In [8]:
parametros_random = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            'max_depth': [1,2,60,70,80,90,100],
            'max_features': [1, 3, 4,5,6,8,10],
            'criterion':['gini','entropy'],
                 }
grid_random = RandomizedSearchCV(
            RandomForestClassifier(), parametros_random,verbose=0, scoring="recall"
        )
grid_random.fit(x_treino, y_treino.values)

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 60, 70, 80, 90,
                                                      100],
                                        'max_features': [1, 3, 4, 5, 6, 8, 10],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   scoring='recall')

In [9]:
rando_model = RandomForestClassifier(**grid_random.best_params_)
rando_model.fit(x_treino, y_treino.values)

RandomForestClassifier(max_depth=70, max_features=6, n_estimators=70)

In [10]:
joblib.dump(rando_model, 'modelos_salvos/random_forest_model.pkl')

['modelos_salvos/random_forest_model.pkl']

## Redes Sequencial

In [23]:
def build_model(hp,camadas_ocultas=4):
    model = keras.Sequential()
    # Primeira camada com input_shape especificado corretamente
    model.add(keras.layers.Dense(
          hp.Choice('units', [8, 16, 32,50,64,100]),
          activation='relu',input_shape=(x_treino.shape[1],)))
    # Camadas ocultas adicionais
    for _ in range(camadas_ocultas):
        model.add(keras.layers.Dense(
            hp.Choice('units', [8, 16, 32,50, 64,100]),
            activation='relu'))
    # Camada de saída
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    # Definição do otimizador
    opt = keras.optimizers.SGD(learning_rate= hp.Choice("learning_rate", [0.001, 0.01, 0.02 ,0.05,0.1, 0.2, 0.3]))
    # Compilação do modelo
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', 'Recall'])
    return model

tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,)

In [24]:
tuner.search(x_treino, y_treino, epochs=10, validation_data=(x_teste, y_teste),verbose=0)
best_model = tuner.get_best_models()[0]

In [26]:
best_model.save("modelos_salvos/redes_neurais_model.h5")